## Homework

In [1]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
id: '=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX -O data.csv': no such user


В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression

In [39]:
data = pd.read_csv("C:\\Users\\Админ\\Desktop\\pandas\\data.csv", parse_dates=["timestamp"])

In [40]:
data.drop("id", axis=1, inplace=True)

In [41]:
columns = list(data.columns)
columns_with_null = []

for column in columns:
    if data[column].isnull().values.any():
        columns_with_null.append(column)
        
print(len(columns_with_null))

51


In [42]:
data_for_cor_matrix_check = pd.DataFrame(data, columns=columns_with_null)
data_for_cor_matrix_check["price"] = data["price"]

corr = data_for_cor_matrix_check.corr()
corr.style.background_gradient(cmap='coolwarm')

,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,preschool_quota,school_quota,hospital_beds_raion,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,metro_min_walk,metro_km_walk,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,cafe_avg_price_2000,cafe_sum_3000_min_price_avg,cafe_sum_3000_max_price_avg,cafe_avg_price_3000,prom_part_5000,cafe_sum_5000_min_price_avg,cafe_sum_5000_max_price_avg,cafe_avg_price_5000,price
life_sq,1.000000,0.119249,0.116271,0.050869,-0.008912,0.553820,0.007038,-0.173120,-0.040999,0.001221,0.000981,-0.017930,-0.108301,-0.011772,0.018146,0.071611,0.066038,-0.095746,-0.001838,-0.010955,0.004770,-0.017899,0.084884,0.012104,-0.054616,-0.090462,0.012292,0.070338,0.070338,0.123870,0.123870,0.026280,0.082625,0.086220,0.085076,0.069116,0.077715,0.074574,0.067617,0.079511,0.075052,0.060491,0.069082,0.065902,0.104410,0.109008,0.107397,-0.138512,0.147551,0.147212,0.147471,0.427630
floor,0.119249,1.000000,0.466684,0.000112,0.001559,-0.008125,-0.009176,-0.117711,0.058213,0.083833,-0.094620,-0.050730,-0.149638,0.029316,0.036984,-0.092455,0.040963,-0.004527,-0.012640,0.015563,0.038017,-0.050768,0.005315,0.000586,-0.152785,-0.015166,0.050733,-0.076887,-0.076887,-0.005285,-0.005285,-0.044888,0.025439,0.034139,0.030810,0.003188,0.019957,0.013383,-0.010895,0.005533,-0.000998,-0.040023,-0.033202,-0.036062,0.034505,0.041649,0.038945,-0.104239,0.026954,0.033731,0.031104,0.119962
max_floor,0.116271,0.466684,1.000000,0.060910,-0.000489,-0.019851,0.022749,-0.081160,0.087119,0.102387,-0.154246,-0.040628,-0.148454,0.057327,0.085981,-0.115507,0.052494,0.004897,0.003903,0.030216,0.058896,-0.040666,-0.001928,0.034559,-0.170459,-0.019350,0.078351,-0.109143,-0.109143,-0.049818,-0.049818,-0.085421,0.100085,0.110221,0.106548,0.047100,0.067944,0.059964,0.008774,0.035562,0.025087,-0.068441,-0.056243,-0.061272,-0.089978,-0.072309,-0.079269,-0.088247,-0.031191,-0.017966,-0.023175,0.084155
material,0.050869,0.000112,0.060910,1.000000,-0.005545,-0.022393,0.044102,-0.033310,-0.137806,-0.121803,0.085555,-0.020900,0.146298,-0.095320,-0.047632,0.082845,0.013855,-0.109682,0.053622,-0.076102,-0.096816,-0.020678,-0.013352,-0.071104,0.107289,-0.111123,-0.064933,0.022536,0.022536,0.037804,0.037804,0.054583,-0.010809,-0.006223,-0.008045,0.017093,0.018755,0.018173,0.010723,0.015740,0.013818,0.055801,0.061409,0.059428,0.060795,0.056063,0.057978,0.073186,0.099847,0.093722,0.096215,0.060041
build_year,-0.008912,0.001559,-0.000489,-0.005545,1.000000,-0.010257,0.000506,0.357205,-0.006354,-0.003474,-0.011037,-0.001136,0.015951,-0.003283,-0.003481,-0.006661,0.002777,0.002869,-0.001510,-0.003430,-0.003835,-0.001126,-0.003111,-0.004301,0.005084,-0.005843,-0.001298,-0.004496,-0.004496,0.003583,0.003583,0.001514,0.002040,0.001957,0.001996,-0.002440,-0.002489,-0.002478,0.001539,0.001052,0.001249,0.002853,0.002400,0.002584,0.000847,0.000698,0.000757,-0.002040,0.000545,0.000500,0.000517,0.002756
num_room,0.553820,-0.008125,-0.019851,-0.022393,-0.010257,1.000000,0.017226,0.072042,-0.006534,0.014946,0.048628,0.027802,0.006904,-0.018080,-0.000071,0.058887,0.062853,0.012057,0.022336,-0.011624,-0.009611,0.027923,0.020687,0.000495,0.019319,0.026731,0.022199,-0.034800,-0.034800,-0.040137,-0.040137,-0.007635,0.009000,0.007173,0.007912,0.017393,0.012248,0.014337,0.029939,0.026482,0.027969,0.031306,0.031255,0.031394,-0.002255,-0.004691,-0.003752,-0.001310,-0.008

Можно убрать следующие признаки, ибо пропусков в них очень много, а с таргетом связи линейной нет: это признаки с наллами больше чем 196 по количеству

Правда есть исключения: life_sq, num_room, но даже если их оставить, то наблюдаеся падение по таргету
Ещё исключение: max_floor, kitc_sq, нужны для инженерии признаков

In [43]:
columns_with_many_nulls = []
columns_with_some_nulls = []

for column in columns_with_null:
    nulls = data[column].isnull().values.sum()
    print(f"in column {column.upper()} {nulls} nulls")
    
    if nulls > 196:
        columns_with_many_nulls.append(column)
    else:
        columns_with_some_nulls.append(column)

in column LIFE_SQ 4103 nulls
in column FLOOR 113 nulls
in column MAX_FLOOR 6303 nulls
in column MATERIAL 6303 nulls
in column BUILD_YEAR 8905 nulls
in column NUM_ROOM 6303 nulls
in column KITCH_SQ 6303 nulls
in column STATE 8907 nulls
in column PRESCHOOL_QUOTA 4279 nulls
in column SCHOOL_QUOTA 4277 nulls
in column HOSPITAL_BEDS_RAION 9404 nulls
in column RAION_BUILD_COUNT_WITH_MATERIAL_INFO 3175 nulls
in column BUILD_COUNT_BLOCK 3175 nulls
in column BUILD_COUNT_WOOD 3175 nulls
in column BUILD_COUNT_FRAME 3175 nulls
in column BUILD_COUNT_BRICK 3175 nulls
in column BUILD_COUNT_MONOLITH 3175 nulls
in column BUILD_COUNT_PANEL 3175 nulls
in column BUILD_COUNT_FOAM 3175 nulls
in column BUILD_COUNT_SLAG 3175 nulls
in column BUILD_COUNT_MIX 3175 nulls
in column RAION_BUILD_COUNT_WITH_BUILDDATE_INFO 3175 nulls
in column BUILD_COUNT_BEFORE_1920 3175 nulls
in column BUILD_COUNT_1921-1945 3175 nulls
in column BUILD_COUNT_1946-1970 3175 nulls
in column BUILD_COUNT_1971-1995 3175 nulls
in column BUI

In [44]:
iskl = ["max_floor", "kitch_sq"]

for col in iskl:
    columns_with_many_nulls.remove(col)
    columns_with_some_nulls.append(col)

In [45]:
for column in columns_with_some_nulls:
    data[column] = data[column].fillna(data[column].mean())

In [46]:
data.drop(columns_with_many_nulls, axis=1, inplace=True)

In [47]:
columns = list(data.columns)
str_features = []

for column in columns:
    if data[column].dtype == "object":
        str_features.append(column)

print(len(str_features))

15


После проверки всех строковых признаков one hot encoding поддаются все, кроме признака "sub_area", его я уберу из данных.

In [48]:
data.drop(["sub_area"], axis=1, inplace=True)

data = pd.get_dummies(data)

Инженерия признаков

In [49]:
month_year = (data.timestamp.dt.month + data.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
data["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (data.timestamp.dt.isocalendar().week + data.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
data["week_year_cnt"] = week_year.map(week_year_cnt_map)

data["month"] = data.timestamp.dt.month
data["day_of_the_week"] = data.timestamp.dt.weekday
data["floor_div_max_floor"] = data["floor"] // data["max_floor"]
data["kitchen_sq_div_full_sq"] = data["kitch_sq"] // data["full_sq"]

In [50]:
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.dropna(inplace=True)

#  почему то в новых признаках по одному пропуску...

In [51]:
data.drop("timestamp", axis=1, inplace=True)

In [74]:
data_invest = data[data["product_type_Investment"] == 1].copy()
data_owner = data[data["product_type_Investment"] == 0].copy()

# Это разделение для задания ниже, его исполнять только для проверки задания на модели для разных
# "product_type"

In [15]:
target = data["price"]
data.drop("price", axis=1, inplace=True)

In [16]:
scaler = StandardScaler()
scaler.fit(data)
data = scaler.transform(data)

Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

Возможно в ваших моделях вам придется указывать, какие колонки являются категориальными (например, в бустингах). Для упрощения предлагается разделить колонки по следующему принципу:
```
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))
```

### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

Добавьте следюущие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitchen_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

Разделите выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [17]:
x_train, x_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42,
                                                    shuffle=True)

In [32]:
lin_reg = LinearRegression().fit(x_train, y_train)
dec_tree = DecisionTreeRegressor().fit(x_train, y_train)
rand_forest = RandomForestRegressor().fit(x_train, y_train)
catboost = CatBoostRegressor().fit(x_train, y_train)

Learning rate set to 0.063276
0:	learn: 4473172.0072020	total: 163ms	remaining: 2m 42s
1:	learn: 4342376.4617933	total: 175ms	remaining: 1m 27s
2:	learn: 4219753.2998358	total: 190ms	remaining: 1m 2s
3:	learn: 4101480.8052886	total: 203ms	remaining: 50.5s
4:	learn: 3999679.0099289	total: 216ms	remaining: 43s
5:	learn: 3898671.7906273	total: 229ms	remaining: 38s
6:	learn: 3809294.7799864	total: 242ms	remaining: 34.3s
7:	learn: 3725309.6958397	total: 255ms	remaining: 31.6s
8:	learn: 3645984.5969876	total: 269ms	remaining: 29.6s
9:	learn: 3576231.3893521	total: 282ms	remaining: 27.9s
10:	learn: 3512892.7695407	total: 295ms	remaining: 26.6s
11:	learn: 3452751.7414032	total: 308ms	remaining: 25.3s
12:	learn: 3392659.7107669	total: 320ms	remaining: 24.3s
13:	learn: 3337594.6546236	total: 332ms	remaining: 23.4s
14:	learn: 3291030.0441820	total: 345ms	remaining: 22.7s
15:	learn: 3249483.1391326	total: 358ms	remaining: 22s
16:	learn: 3205846.3379017	total: 372ms	remaining: 21.5s
17:	learn: 3171

157:	learn: 2304731.6268142	total: 2.18s	remaining: 11.6s
158:	learn: 2304064.0842857	total: 2.2s	remaining: 11.6s
159:	learn: 2302706.0078852	total: 2.21s	remaining: 11.6s
160:	learn: 2300907.1972692	total: 2.22s	remaining: 11.6s
161:	learn: 2297719.3375619	total: 2.23s	remaining: 11.6s
162:	learn: 2297109.3874352	total: 2.25s	remaining: 11.5s
163:	learn: 2294542.7885986	total: 2.26s	remaining: 11.5s
164:	learn: 2292404.8396607	total: 2.27s	remaining: 11.5s
165:	learn: 2290543.9876782	total: 2.28s	remaining: 11.5s
166:	learn: 2289805.3015612	total: 2.29s	remaining: 11.4s
167:	learn: 2287940.8166443	total: 2.31s	remaining: 11.4s
168:	learn: 2284850.3195053	total: 2.32s	remaining: 11.4s
169:	learn: 2283659.5499155	total: 2.33s	remaining: 11.4s
170:	learn: 2282510.5308769	total: 2.35s	remaining: 11.4s
171:	learn: 2279778.7600469	total: 2.36s	remaining: 11.4s
172:	learn: 2278873.5914272	total: 2.37s	remaining: 11.3s
173:	learn: 2276542.8888510	total: 2.38s	remaining: 11.3s
174:	learn: 227

314:	learn: 2097175.9958362	total: 4.13s	remaining: 8.99s
315:	learn: 2096149.1815303	total: 4.15s	remaining: 8.98s
316:	learn: 2094631.7521341	total: 4.16s	remaining: 8.96s
317:	learn: 2093177.5078992	total: 4.17s	remaining: 8.95s
318:	learn: 2092239.3584145	total: 4.19s	remaining: 8.94s
319:	learn: 2091091.3164325	total: 4.2s	remaining: 8.92s
320:	learn: 2090215.7147554	total: 4.21s	remaining: 8.91s
321:	learn: 2089256.2727890	total: 4.22s	remaining: 8.89s
322:	learn: 2088075.3361191	total: 4.24s	remaining: 8.88s
323:	learn: 2087229.9634431	total: 4.25s	remaining: 8.86s
324:	learn: 2086510.7331527	total: 4.26s	remaining: 8.85s
325:	learn: 2085842.9254663	total: 4.27s	remaining: 8.83s
326:	learn: 2084859.4679819	total: 4.29s	remaining: 8.82s
327:	learn: 2083546.5516265	total: 4.3s	remaining: 8.8s
328:	learn: 2081735.0806259	total: 4.31s	remaining: 8.79s
329:	learn: 2080252.3739138	total: 4.32s	remaining: 8.78s
330:	learn: 2079049.3612414	total: 4.34s	remaining: 8.77s
331:	learn: 20776

459:	learn: 1965556.0705976	total: 5.95s	remaining: 6.99s
460:	learn: 1963998.9363371	total: 5.97s	remaining: 6.98s
461:	learn: 1962881.7237233	total: 5.98s	remaining: 6.96s
462:	learn: 1961992.9373337	total: 5.99s	remaining: 6.95s
463:	learn: 1961001.0220083	total: 6s	remaining: 6.94s
464:	learn: 1959835.9468164	total: 6.02s	remaining: 6.92s
465:	learn: 1958702.0316142	total: 6.03s	remaining: 6.91s
466:	learn: 1958039.9636724	total: 6.04s	remaining: 6.9s
467:	learn: 1957986.0557145	total: 6.06s	remaining: 6.88s
468:	learn: 1957488.2532537	total: 6.07s	remaining: 6.87s
469:	learn: 1956546.4401708	total: 6.08s	remaining: 6.86s
470:	learn: 1955949.2430033	total: 6.09s	remaining: 6.84s
471:	learn: 1955026.0728849	total: 6.11s	remaining: 6.83s
472:	learn: 1954112.7620374	total: 6.12s	remaining: 6.82s
473:	learn: 1954074.1896183	total: 6.13s	remaining: 6.8s
474:	learn: 1953205.2280281	total: 6.14s	remaining: 6.79s
475:	learn: 1952624.5818264	total: 6.16s	remaining: 6.78s
476:	learn: 1951676

614:	learn: 1828328.6315827	total: 7.93s	remaining: 4.96s
615:	learn: 1827999.5613881	total: 7.94s	remaining: 4.95s
616:	learn: 1827115.0722026	total: 7.95s	remaining: 4.94s
617:	learn: 1825739.8101649	total: 7.97s	remaining: 4.92s
618:	learn: 1824554.0817112	total: 7.98s	remaining: 4.91s
619:	learn: 1824179.7503795	total: 7.99s	remaining: 4.9s
620:	learn: 1823967.0046052	total: 8s	remaining: 4.88s
621:	learn: 1822986.2138038	total: 8.01s	remaining: 4.87s
622:	learn: 1822183.2728434	total: 8.03s	remaining: 4.86s
623:	learn: 1821128.0705099	total: 8.04s	remaining: 4.84s
624:	learn: 1820652.7463560	total: 8.05s	remaining: 4.83s
625:	learn: 1820216.7468761	total: 8.06s	remaining: 4.82s
626:	learn: 1820186.8865131	total: 8.07s	remaining: 4.8s
627:	learn: 1819509.2609226	total: 8.09s	remaining: 4.79s
628:	learn: 1818434.4729971	total: 8.1s	remaining: 4.78s
629:	learn: 1817145.8926434	total: 8.11s	remaining: 4.76s
630:	learn: 1816222.2027001	total: 8.13s	remaining: 4.75s
631:	learn: 1815264.

758:	learn: 1745369.2964130	total: 9.73s	remaining: 3.09s
759:	learn: 1744678.6038236	total: 9.74s	remaining: 3.08s
760:	learn: 1743951.0847977	total: 9.76s	remaining: 3.06s
761:	learn: 1743230.6442564	total: 9.77s	remaining: 3.05s
762:	learn: 1742969.8487463	total: 9.78s	remaining: 3.04s
763:	learn: 1742479.9443439	total: 9.79s	remaining: 3.02s
764:	learn: 1741993.0251074	total: 9.8s	remaining: 3.01s
765:	learn: 1741778.6818834	total: 9.82s	remaining: 3s
766:	learn: 1740901.8624083	total: 9.83s	remaining: 2.98s
767:	learn: 1740416.2552587	total: 9.84s	remaining: 2.97s
768:	learn: 1740010.1218573	total: 9.85s	remaining: 2.96s
769:	learn: 1739494.4806945	total: 9.87s	remaining: 2.95s
770:	learn: 1738817.1938352	total: 9.88s	remaining: 2.93s
771:	learn: 1738457.6586755	total: 9.89s	remaining: 2.92s
772:	learn: 1737651.4507284	total: 9.9s	remaining: 2.91s
773:	learn: 1737137.0815057	total: 9.91s	remaining: 2.9s
774:	learn: 1736072.7326013	total: 9.93s	remaining: 2.88s
775:	learn: 1735475.

912:	learn: 1652440.6468929	total: 11.7s	remaining: 1.11s
913:	learn: 1651901.3741333	total: 11.7s	remaining: 1.1s
914:	learn: 1651737.9129103	total: 11.7s	remaining: 1.09s
915:	learn: 1651306.0632015	total: 11.7s	remaining: 1.08s
916:	learn: 1650748.7829587	total: 11.7s	remaining: 1.06s
917:	learn: 1650016.7547018	total: 11.8s	remaining: 1.05s
918:	learn: 1649265.2247776	total: 11.8s	remaining: 1.04s
919:	learn: 1648724.1528154	total: 11.8s	remaining: 1.02s
920:	learn: 1648294.3465103	total: 11.8s	remaining: 1.01s
921:	learn: 1647920.6269796	total: 11.8s	remaining: 999ms
922:	learn: 1647748.3017377	total: 11.8s	remaining: 986ms
923:	learn: 1647287.9739700	total: 11.8s	remaining: 973ms
924:	learn: 1646869.1005975	total: 11.8s	remaining: 960ms
925:	learn: 1645949.8215438	total: 11.9s	remaining: 948ms
926:	learn: 1645403.5363298	total: 11.9s	remaining: 935ms
927:	learn: 1644389.6833509	total: 11.9s	remaining: 922ms
928:	learn: 1643858.6463361	total: 11.9s	remaining: 909ms
929:	learn: 164

In [33]:
y_pred_lin_reg = lin_reg.predict(x_test)
y_pred_dec_tree = dec_tree.predict(x_test)
y_pred_rand_forest = rand_forest.predict(x_test)
y_pred_catboost = catboost.predict(x_test)

print(f"RMSE на линейной регрессии: {int(mean_squared_error(y_test, y_pred_lin_reg, squared=False))}")
print(f"RMSE на обычном дереве: {int(mean_squared_error(y_test, y_pred_dec_tree, squared=False))}")
print(f"RMSE на случайном лесу: {int(mean_squared_error(y_test, y_pred_rand_forest, squared=False))}")
print(f"RMSE на catboost: {int(mean_squared_error(y_test, y_pred_catboost, squared=False))}")

RMSE на линейной регрессии: 3909760
RMSE на обычном дереве: 3611136
RMSE на случайном лесу: 2613359
RMSE на catboost: 2543949


### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

GridSearchCV не дал прироста, наоборот же дал падение на метрике. Лучшая модель, очевидно, catboost. Но также есть и случайный лес, который показал результат чуть хуже чем catboost. 

### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

In [75]:
target_invest = data_invest["price"]
data_invest.drop("price", axis=1, inplace=True)

target_owner = data_owner["price"]
data_owner.drop("price", axis=1, inplace=True)

In [78]:
scaler = StandardScaler()
scaler.fit(data_invest)
data_invest = scaler.transform(data_invest)

scaler = StandardScaler()
scaler.fit(data_invest)
data_invest = scaler.transform(data_invest)

In [79]:
x_train_invest, x_test_invest, y_train_invest, y_test_invest = train_test_split(data_invest, target_invest, 
                                                                                test_size=0.2, 
                                                                                random_state=42,
                                                                                shuffle=True)

x_train_owner, x_test_owner, y_train_owner, y_test_owner = train_test_split(data_owner, target_owner, 
                                                                                test_size=0.2, 
                                                                                random_state=42,
                                                                                shuffle=True)

In [80]:
x_test = np.concatenate((x_test_invest, x_test_owner), axis=0)
y_test = np.concatenate((y_test_invest, y_test_owner), axis=0)

In [81]:
model_invest = CatBoostRegressor().fit(x_train_invest, y_train_invest)
model_owner = CatBoostRegressor().fit(x_train_owner, y_train_owner)

Learning rate set to 0.059158
0:	learn: 4802227.9243477	total: 11.7ms	remaining: 11.7s
1:	learn: 4678788.4904565	total: 23.5ms	remaining: 11.7s
2:	learn: 4565967.4544160	total: 35.2ms	remaining: 11.7s
3:	learn: 4450123.4817538	total: 46.8ms	remaining: 11.6s
4:	learn: 4344878.5564883	total: 58.7ms	remaining: 11.7s
5:	learn: 4245061.9563664	total: 71.2ms	remaining: 11.8s
6:	learn: 4161679.3384985	total: 83.3ms	remaining: 11.8s
7:	learn: 4077332.7383366	total: 95.3ms	remaining: 11.8s
8:	learn: 3995421.7280657	total: 107ms	remaining: 11.8s
9:	learn: 3922573.4473030	total: 119ms	remaining: 11.7s
10:	learn: 3854087.9775408	total: 130ms	remaining: 11.7s
11:	learn: 3795663.3453356	total: 141ms	remaining: 11.6s
12:	learn: 3741664.0576643	total: 153ms	remaining: 11.6s
13:	learn: 3688642.4550409	total: 165ms	remaining: 11.6s
14:	learn: 3642384.8591593	total: 177ms	remaining: 11.6s
15:	learn: 3596078.2210940	total: 190ms	remaining: 11.7s
16:	learn: 3552666.8640345	total: 203ms	remaining: 11.7s
17:

148:	learn: 2617469.9307093	total: 1.77s	remaining: 10.1s
149:	learn: 2615506.4474209	total: 1.79s	remaining: 10.1s
150:	learn: 2613442.3209119	total: 1.8s	remaining: 10.1s
151:	learn: 2611211.8066552	total: 1.81s	remaining: 10.1s
152:	learn: 2609093.3560677	total: 1.82s	remaining: 10.1s
153:	learn: 2606691.6787394	total: 1.83s	remaining: 10.1s
154:	learn: 2604786.0801309	total: 1.84s	remaining: 10.1s
155:	learn: 2603351.0078442	total: 1.86s	remaining: 10s
156:	learn: 2600637.0348826	total: 1.87s	remaining: 10s
157:	learn: 2598411.8653290	total: 1.88s	remaining: 10s
158:	learn: 2596498.2957902	total: 1.89s	remaining: 10s
159:	learn: 2593946.9711040	total: 1.9s	remaining: 10s
160:	learn: 2592480.7835172	total: 1.92s	remaining: 9.98s
161:	learn: 2590604.8587259	total: 1.93s	remaining: 9.97s
162:	learn: 2589831.8873451	total: 1.94s	remaining: 9.96s
163:	learn: 2587308.8420470	total: 1.95s	remaining: 9.94s
164:	learn: 2586150.9270071	total: 1.96s	remaining: 9.92s
165:	learn: 2584514.870787

298:	learn: 2373536.9294711	total: 3.54s	remaining: 8.31s
299:	learn: 2372047.9742741	total: 3.56s	remaining: 8.3s
300:	learn: 2369773.2001648	total: 3.57s	remaining: 8.29s
301:	learn: 2367873.2430561	total: 3.58s	remaining: 8.28s
302:	learn: 2365828.1038986	total: 3.59s	remaining: 8.27s
303:	learn: 2363254.1482100	total: 3.6s	remaining: 8.25s
304:	learn: 2361971.4918335	total: 3.62s	remaining: 8.24s
305:	learn: 2360171.2964002	total: 3.63s	remaining: 8.23s
306:	learn: 2358858.8397049	total: 3.64s	remaining: 8.22s
307:	learn: 2357050.0879459	total: 3.65s	remaining: 8.2s
308:	learn: 2356976.0973432	total: 3.66s	remaining: 8.19s
309:	learn: 2355390.9668211	total: 3.67s	remaining: 8.18s
310:	learn: 2353831.4317114	total: 3.69s	remaining: 8.17s
311:	learn: 2352736.5519065	total: 3.7s	remaining: 8.16s
312:	learn: 2350712.1918937	total: 3.71s	remaining: 8.14s
313:	learn: 2349412.6803056	total: 3.72s	remaining: 8.13s
314:	learn: 2347719.4244842	total: 3.73s	remaining: 8.12s
315:	learn: 234642

450:	learn: 2193955.9516557	total: 5.33s	remaining: 6.5s
451:	learn: 2192431.4304817	total: 5.35s	remaining: 6.48s
452:	learn: 2191723.0260749	total: 5.36s	remaining: 6.47s
453:	learn: 2190022.0484761	total: 5.37s	remaining: 6.46s
454:	learn: 2189079.5969944	total: 5.38s	remaining: 6.45s
455:	learn: 2188439.7348269	total: 5.39s	remaining: 6.43s
456:	learn: 2187583.3092248	total: 5.4s	remaining: 6.42s
457:	learn: 2186309.1060804	total: 5.42s	remaining: 6.41s
458:	learn: 2185980.4389431	total: 5.43s	remaining: 6.4s
459:	learn: 2185156.5294838	total: 5.44s	remaining: 6.38s
460:	learn: 2183255.9286428	total: 5.45s	remaining: 6.37s
461:	learn: 2182236.7797496	total: 5.46s	remaining: 6.36s
462:	learn: 2180945.9564700	total: 5.47s	remaining: 6.35s
463:	learn: 2180090.3436067	total: 5.48s	remaining: 6.33s
464:	learn: 2178603.5304755	total: 5.5s	remaining: 6.32s
465:	learn: 2177459.3108246	total: 5.51s	remaining: 6.31s
466:	learn: 2176776.3796372	total: 5.52s	remaining: 6.3s
467:	learn: 2174967

598:	learn: 2052601.9668062	total: 7.11s	remaining: 4.76s
599:	learn: 2051460.4140921	total: 7.13s	remaining: 4.75s
600:	learn: 2050229.7789603	total: 7.14s	remaining: 4.74s
601:	learn: 2049051.3131874	total: 7.15s	remaining: 4.73s
602:	learn: 2047980.3983538	total: 7.16s	remaining: 4.71s
603:	learn: 2047070.7715903	total: 7.17s	remaining: 4.7s
604:	learn: 2045784.5085738	total: 7.18s	remaining: 4.69s
605:	learn: 2045085.8675226	total: 7.2s	remaining: 4.68s
606:	learn: 2044390.6582012	total: 7.21s	remaining: 4.67s
607:	learn: 2043106.8292255	total: 7.22s	remaining: 4.65s
608:	learn: 2042806.0309878	total: 7.23s	remaining: 4.64s
609:	learn: 2041289.9745475	total: 7.24s	remaining: 4.63s
610:	learn: 2040640.6594977	total: 7.25s	remaining: 4.62s
611:	learn: 2039991.3303314	total: 7.26s	remaining: 4.61s
612:	learn: 2039016.4700230	total: 7.28s	remaining: 4.59s
613:	learn: 2037334.8593008	total: 7.29s	remaining: 4.58s
614:	learn: 2035736.1323470	total: 7.3s	remaining: 4.57s
615:	learn: 20350

...

350:	learn: 791319.4774526	total: 3.02s	remaining: 5.58s
351:	learn: 791176.8454859	total: 3.03s	remaining: 5.57s
352:	learn: 790909.0440903	total: 3.04s	remaining: 5.56s
353:	learn: 789924.6888467	total: 3.04s	remaining: 5.56s
354:	learn: 788756.7108490	total: 3.05s	remaining: 5.55s
355:	learn: 787860.7587472	total: 3.06s	remaining: 5.54s
356:	learn: 786849.4156268	total: 3.07s	remaining: 5.53s
357:	learn: 786544.1555584	total: 3.08s	remaining: 5.52s
358:	learn: 786315.8142702	total: 3.09s	remaining: 5.51s
359:	learn: 785440.7725273	total: 3.1s	remaining: 5.5s
360:	learn: 783835.7469774	total: 3.1s	remaining: 5.5s
361:	learn: 783137.4060566	total: 3.11s	remaining: 5.49s
362:	learn: 782890.7333444	total: 3.12s	remaining: 5.48s
363:	learn: 780849.8915043	total: 3.13s	remaining: 5.47s
364:	learn: 778413.2508397	total: 3.14s	remaining: 5.46s
365:	learn: 777689.8546094	total: 3.15s	remaining: 5.45s
366:	learn: 776518.4010146	total: 3.16s	remaining: 5.44s
367:	learn: 775537.7236126	total: 3

504:	learn: 674150.7837681	total: 4.36s	remaining: 4.28s
505:	learn: 674029.5251384	total: 4.37s	remaining: 4.27s
506:	learn: 673679.3444166	total: 4.38s	remaining: 4.26s
507:	learn: 672649.8011200	total: 4.39s	remaining: 4.25s
508:	learn: 672476.4924238	total: 4.4s	remaining: 4.24s
509:	learn: 671602.2403350	total: 4.41s	remaining: 4.23s
510:	learn: 671273.5561431	total: 4.41s	remaining: 4.22s
511:	learn: 670855.3919065	total: 4.42s	remaining: 4.21s
512:	learn: 669634.2205445	total: 4.43s	remaining: 4.21s
513:	learn: 668284.7582736	total: 4.44s	remaining: 4.2s
514:	learn: 668175.2327470	total: 4.45s	remaining: 4.19s
515:	learn: 667251.1804014	total: 4.46s	remaining: 4.18s
516:	learn: 666721.6847973	total: 4.47s	remaining: 4.17s
517:	learn: 665747.0877688	total: 4.47s	remaining: 4.16s
518:	learn: 664576.3508597	total: 4.48s	remaining: 4.16s
519:	learn: 663679.9130997	total: 4.49s	remaining: 4.14s
520:	learn: 662377.1928267	total: 4.5s	remaining: 4.14s
521:	learn: 661816.2675729	total: 

658:	learn: 597732.1212888	total: 5.7s	remaining: 2.95s
659:	learn: 597467.0482728	total: 5.71s	remaining: 2.94s
660:	learn: 597252.6834187	total: 5.71s	remaining: 2.93s
661:	learn: 597143.0236152	total: 5.72s	remaining: 2.92s
662:	learn: 596350.7051178	total: 5.73s	remaining: 2.91s
663:	learn: 595768.6188956	total: 5.74s	remaining: 2.9s
664:	learn: 595171.3167594	total: 5.75s	remaining: 2.9s
665:	learn: 595135.7153883	total: 5.76s	remaining: 2.89s
666:	learn: 595104.2341041	total: 5.77s	remaining: 2.88s
667:	learn: 594977.4095333	total: 5.78s	remaining: 2.87s
668:	learn: 594946.8608524	total: 5.78s	remaining: 2.86s
669:	learn: 594920.5492304	total: 5.79s	remaining: 2.85s
670:	learn: 594464.3602538	total: 5.8s	remaining: 2.84s
671:	learn: 594221.2848424	total: 5.81s	remaining: 2.83s
672:	learn: 594064.7165090	total: 5.82s	remaining: 2.83s
673:	learn: 592657.8722685	total: 5.83s	remaining: 2.82s
674:	learn: 592204.6187420	total: 5.83s	remaining: 2.81s
675:	learn: 591992.1646803	total: 5

821:	learn: 528552.4054729	total: 7.19s	remaining: 1.56s
822:	learn: 528191.3187599	total: 7.2s	remaining: 1.55s
823:	learn: 527889.1167467	total: 7.21s	remaining: 1.54s
824:	learn: 527482.1208979	total: 7.22s	remaining: 1.53s
825:	learn: 527360.1168386	total: 7.23s	remaining: 1.52s
826:	learn: 527096.1469403	total: 7.24s	remaining: 1.51s
827:	learn: 526969.4884452	total: 7.25s	remaining: 1.5s
828:	learn: 526652.7139422	total: 7.25s	remaining: 1.5s
829:	learn: 526375.3336855	total: 7.26s	remaining: 1.49s
830:	learn: 526194.8497311	total: 7.27s	remaining: 1.48s
831:	learn: 526157.6199156	total: 7.28s	remaining: 1.47s
832:	learn: 526139.9257272	total: 7.29s	remaining: 1.46s
833:	learn: 526096.1741664	total: 7.29s	remaining: 1.45s
834:	learn: 525581.5530927	total: 7.3s	remaining: 1.44s
835:	learn: 525564.3356206	total: 7.31s	remaining: 1.43s
836:	learn: 524910.3994002	total: 7.32s	remaining: 1.43s
837:	learn: 524341.0485408	total: 7.33s	remaining: 1.42s
838:	learn: 523726.0851752	total: 7

967:	learn: 482967.8275735	total: 8.48s	remaining: 280ms
968:	learn: 482618.5777049	total: 8.49s	remaining: 272ms
969:	learn: 481891.3527848	total: 8.5s	remaining: 263ms
970:	learn: 481234.3401135	total: 8.51s	remaining: 254ms
971:	learn: 481218.8734194	total: 8.52s	remaining: 245ms
972:	learn: 480977.7621535	total: 8.53s	remaining: 237ms
973:	learn: 480487.4305484	total: 8.53s	remaining: 228ms
974:	learn: 480252.0766403	total: 8.54s	remaining: 219ms
975:	learn: 480023.0935070	total: 8.55s	remaining: 210ms
976:	learn: 479736.7433176	total: 8.56s	remaining: 202ms
977:	learn: 479458.6852663	total: 8.58s	remaining: 193ms
978:	learn: 479391.6871001	total: 8.59s	remaining: 184ms
979:	learn: 479136.7754622	total: 8.59s	remaining: 175ms
980:	learn: 478810.2257638	total: 8.6s	remaining: 167ms
981:	learn: 478668.4891573	total: 8.61s	remaining: 158ms
982:	learn: 478305.3984675	total: 8.62s	remaining: 149ms
983:	learn: 477818.0656603	total: 8.63s	remaining: 140ms
984:	learn: 477588.1794892	total:

In [82]:
y_pred_model_invest = model_invest.predict(x_test)
y_pred_model_owner = model_owner.predict(x_test)

In [83]:
print(f"RMSE на catboost invest: {int(mean_squared_error(y_test, y_pred_model_invest, squared=False))}")
print(f"RMSE на catboost owner : {int(mean_squared_error(y_test, y_pred_model_owner, squared=False))}")

RMSE на catboost invest: 10805228
RMSE на catboost owner : 3810545


### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).